# AgentCore リソースのクリーンアップ

このノートブックは、ラボ演習中に作成された AgentCore Memory と Gateway リソースをクリーンアップします。

**⚠️ 重要:** このノートブックは、ラボ手順の**タスク 2** を完了した後にのみ実行してください。タスク 2 では AgentCore ダッシュボードの探索が含まれます。タスク 2 の前にこのクリーンアップを実行すると、探索に必要なリソースが削除されます。

## このノートブックが行うこと

このクリーンアッププロセスは以下を実行します:
1. すべての AgentCore Memory リソースを削除
2. すべての AgentCore Gateway ターゲットを削除
3. すべての AgentCore Gateway リソースを削除

**注意:** このクリーンアップは以下を削除しません:
- Lambda 関数(CloudFormation で管理)
- DynamoDB テーブル(CloudFormation で管理)
- ラボの CloudFormation スタックによって作成されたその他の AWS リソース

これらのリソースは、ラボの最後に CloudFormation スタックを削除するときにクリーンアップされます。

## セットアップ

必要なライブラリをインポートし、AWS リージョンを設定します。

In [ ]:
import boto3
import time
from botocore.exceptions import ClientError

# 現在の AWS リージョンを取得
session = boto3.session.Session()
REGION = session.region_name
print(f"Using AWS Region: {REGION}")

## AgentCore リソースの削除

以下のセルを実行して、すべての AgentCore Memory と Gateway リソースを削除します。

このプロセスは以下を実行します:
1. すべての Memory リソースをリストして削除
2. すべての Gateway ターゲットをリストして削除(Gateway の前に削除する必要があります)
3. ターゲットの削除が完了するまで待機
4. すべての Gateway リソースをリストして削除

In [ ]:
# AgentCore クライアントを初期化
agentcore_client = boto3.client("bedrock-agentcore-control", region_name=REGION)

print("Starting AgentCore resource cleanup...\n")

# Memory リソースを削除
print("🗑️  Deleting Memory resources...")
try:
    memories = agentcore_client.list_memories()['memories']
    
    if not memories:
        print("   No Memory resources found.")
    else:
        for memory in memories:
            memory_id = memory['id']
            memory_name = memory.get('name', 'Unnamed')
            try:
                agentcore_client.delete_memory(memoryId=memory_id)
                print(f"   ✓ Deleted Memory: {memory_name} (ID: {memory_id})")
            except ClientError as e:
                print(f"   ✗ Failed to delete Memory {memory_name}: {e.response['Error']['Message']}")
except ClientError as e:
    print(f"   ✗ Error listing memories: {e.response['Error']['Message']}")

print("\n🗑️  Deleting Gateway resources...")

# Gateway リソースを削除
try:
    gateways = agentcore_client.list_gateways()['items']
    
    if not gateways:
        print("   No Gateway resources found.")
    else:
        for gateway in gateways:
            gateway_id = gateway['gatewayId']
            gateway_name = gateway.get('name', 'Unnamed')
            
            # 最初に、すべての Gateway ターゲットを削除
            try:
                gateway_targets = agentcore_client.list_gateway_targets(
                    gatewayIdentifier=gateway_id
                )['items']
                
                if gateway_targets:
                    print(f"   Deleting targets for Gateway: {gateway_name}")
                    for gateway_target in gateway_targets:
                        target_id = gateway_target['targetId']
                        try:
                            agentcore_client.delete_gateway_target(
                                gatewayIdentifier=gateway_id,
                                targetId=target_id
                            )
                            print(f"      ✓ Deleted Gateway target: {target_id}")
                        except ClientError as e:
                            print(f"      ✗ Failed to delete target {target_id}: {e.response['Error']['Message']}")
                    
                    # ターゲットの削除が完了するまで待機
                    print(f"   Waiting for target deletion to complete...")
                    max_wait = 60  # 最大待機時間(秒)
                    wait_interval = 2  # 2秒ごとにチェック
                    elapsed = 0
                    
                    while elapsed < max_wait:
                        try:
                            remaining_targets = agentcore_client.list_gateway_targets(
                                gatewayIdentifier=gateway_id
                            )['items']
                            if not remaining_targets:
                                print(f"   ✓ All targets deleted successfully")
                                break
                            time.sleep(wait_interval)
                            elapsed += wait_interval
                        except ClientError:
                            break
                    
                    if elapsed >= max_wait:
                        print(f"   ⚠️  Warning: Target deletion timed out after {max_wait}s")
                
                # 次に Gateway 自体を削除
                try:
                    agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)
                    print(f"   ✓ Deleted Gateway: {gateway_name} (ID: {gateway_id})")
                except ClientError as e:
                    print(f"   ✗ Failed to delete Gateway {gateway_name}: {e.response['Error']['Message']}")
                    
            except ClientError as e:
                print(f"   ✗ Error processing Gateway {gateway_name}: {e.response['Error']['Message']}")
                
except ClientError as e:
    print(f"   ✗ Error listing gateways: {e.response['Error']['Message']}")

print("\n✅ AgentCore resource cleanup complete!")
print("\nYou can now close this notebook and return to the lab instructions.")